In [1]:
import os
import shutil
from PIL import Image
import imagehash

def find_and_handle_duplicates(folder_A: str,
                               folder_B: str,
                               first_only: bool = True,
                               threshold: int = 5,
                               output_base: str = "output"):
    """
    Busca duplicados entre dos carpetas usando perceptual hash (pHash).
    
    Parámetros:
    - folder_A, folder_B: rutas a los dos datasets
    - first_only: 
        * True  → retorna la primera pareja duplicada (fname_A, fname_B)
        * False → organiza archivos en output_base:
            - duplicados/…  (subcarpeta por pareja)
            - new_dataset/dataset_A/ (únicos de A)
            - new_dataset/dataset_B/ (únicos de B)
    - threshold: distancia máxima de Hamming para considerar “duplicado”
    - output_base: carpeta raíz donde se creará la salida
    
    Retorna (solo si first_only=True):
      (fname_A, fname_B, dist)
    """
    # 1) Construir índice de hashes para cada carpeta
    def build_index(folder):
        idx = {}
        for fn in os.listdir(folder):
            path = os.path.join(folder, fn)
            try:
                idx[fn] = imagehash.phash(Image.open(path))
            except Exception as e:
                print(f"⚠️ Skipping {fn}: {e}")
        return idx

    hash_A = build_index(folder_A)
    hash_B = build_index(folder_B)

    # 2) Si solo queremos la primera pareja, devolvemos y salimos
    for fa, ha in hash_A.items():
        for fb, hb in hash_B.items():
            dist = abs(ha - hb)
            if dist <= threshold:
                if first_only:
                    return fa, fb, dist

    if first_only:
        return None  # no se encontró ninguna pareja

    # 3) Si vamos a organizar todo:
    #    - crear carpetas de salida
    dup_base = os.path.join(output_base, "duplicados")
    uniq_base = os.path.join(output_base, "new_dataset")
    os.makedirs(dup_base, exist_ok=True)
    os.makedirs(os.path.join(uniq_base, "dataset_A"), exist_ok=True)
    os.makedirs(os.path.join(uniq_base, "dataset_B"), exist_ok=True)

    # 4) detectar todas las parejas duplicadas
    duplicated_A = set()
    duplicated_B = set()
    pairings = []
    for fa, ha in hash_A.items():
        for fb, hb in hash_B.items():
            if abs(ha - hb) <= threshold:
                pairings.append((fa, fb))
                duplicated_A.add(fa)
                duplicated_B.add(fb)

    # 5) copiar duplicados en subcarpetas numeradas
    for i, (fa, fb) in enumerate(pairings, start=1):
        sub = os.path.join(dup_base, f"pair_{i}")
        os.makedirs(sub, exist_ok=True)
        shutil.copy2(os.path.join(folder_A, fa), sub)
        shutil.copy2(os.path.join(folder_B, fb), sub)

    # 6) copiar únicos restantes
    for fa in hash_A:
        if fa not in duplicated_A:
            shutil.copy2(os.path.join(folder_A, fa),
                         os.path.join(uniq_base, "dataset_A", fa))
    for fb in hash_B:
        if fb not in duplicated_B:
            shutil.copy2(os.path.join(folder_B, fb),
                         os.path.join(uniq_base, "dataset_B", fb))

    print(f"✅ Copiadas {len(pairings)} parejas duplicadas en '{dup_base}'")
    print(f"✅ Imágenes únicas de A en '{os.path.join(uniq_base, 'dataset_A')}'")
    print(f"✅ Imágenes únicas de B en '{os.path.join(uniq_base, 'dataset_B')}'")


In [ ]:
# 1) Solo la primera pareja duplicada
res = find_and_handle_duplicates("CellData/chest_xray/train/NORMAL", "Data/chest_xray/train/NORMAL", first_only=True)
if res:
    fa, fb, dist = res
    print("Primera pareja duplicada:", fa, fb, "dist=", dist)
else:
    print("No se encontró duplicado")

# 2) Organizar TODO en subcarpetas
find_and_handle_duplicates("CellData/chest_xray/train/NORMAL", "Data/chest_xray/train/NORMAL", first_only=False, threshold=5, output_base="resultados-neumonia")



Primera pareja duplicada: NORMAL-1003233-0001.jpeg IM-0734-0001.jpeg dist= 0
✅ Copiadas 1651 parejas duplicadas en 'resultados\duplicados'
✅ Imágenes únicas de A en 'resultados\new_dataset\dataset_A'
✅ Imágenes únicas de B en 'resultados\new_dataset\dataset_B'


In [6]:
# 1) Solo la primera pareja duplicada
res = find_and_handle_duplicates("CellData/chest_xray/train/PNEUMONIA", "Data/chest_xray/train/PNEUMONIA", first_only=True)
if res:
    fa, fb, dist = res
    print("Primera pareja duplicada:", fa, fb, "dist=", dist)
else:
    print("No se encontró duplicado")

# 2) Organizar TODO en subcarpetas
find_and_handle_duplicates("CellData/chest_xray/train/PNEUMONIA", "Data/chest_xray/train/PNEUMONIA", first_only=False, threshold=5, output_base="resultados-neumonia")



Primera pareja duplicada: BACTERIA-1008087-0001.jpeg person986_bacteria_2913.jpeg dist= 0
✅ Copiadas 3967 parejas duplicadas en 'resultados-neumonia\duplicados'
✅ Imágenes únicas de A en 'resultados-neumonia\new_dataset\dataset_A'
✅ Imágenes únicas de B en 'resultados-neumonia\new_dataset\dataset_B'
